In [1]:
import os
import pandas as pd
import requests

In [2]:
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop

2024-11-15 09:32:39.152643: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 09:32:39.197576: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 09:32:40.933940: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 09:32:41.213394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731681161.658643   89553 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731681161.83

In [3]:
# Load the CSV data
train_df = pd.read_csv('train_updated_book_covers.csv')
test_df = pd.read_csv('test_updated_book_covers.csv')

# Split the training data into train and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.15, random_state=42, stratify=train_df['Genre'])

In [4]:
# Directory to save downloaded images
image_dir = 'book_cover_images'
os.makedirs(image_dir, exist_ok=True)

# Function to download an image and save it locally
def download_image(url, save_path):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                for chunk in response:
                    f.write(chunk)
        return save_path
    except Exception as e:
        print(f"Failed to download {url}: {e}")
        return None

# Download each image and save the local path in the DataFrame
train_df['Local Image Path'] = [
    download_image(url, os.path.join(image_dir, f"book_cover_{idx}.jpg"))
    for idx, url in tqdm(enumerate(train_df['Book Cover Image URL']))
]

val_df['Local Image Path'] = [
    download_image(url, os.path.join(image_dir, f"val_book_cover_{idx}.jpg"))
    for idx, url in tqdm(enumerate(val_df['Book Cover Image URL']))
]

test_df['Local Image Path'] = [
    download_image(url, os.path.join(image_dir, f"test_book_cover_{idx}.jpg"))
    for idx, url in tqdm(enumerate(test_df['Book Cover Image URL']))
]


1087it [00:53, 20.40it/s]
192it [00:09, 19.68it/s]
260it [00:10, 25.59it/s]


In [5]:
#print(test_df)

In [6]:
# Define image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only rescale validation and test data
val_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load training data from the DataFrame
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Local Image Path',  # Use local paths
    y_col='Genre',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load validation data from the DataFrame
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='Local Image Path',
    y_col='Genre',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load test data from the DataFrame
test_generator = val_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Local Image Path',
    y_col='Genre',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 1087 validated image filenames belonging to 13 classes.
Found 192 validated image filenames belonging to 13 classes.
Found 260 validated image filenames belonging to 13 classes.


In [7]:
# Load pre-trained ResNet50 without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze ResNet50 layers


#numclasses manually:
num_classes = len(train_generator.class_indices)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
#predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # Output layer for genres
predictions = Dense(num_classes, activation='softmax')(x)
# Build the model
model = Model(inputs=base_model.input, outputs=predictions)


2024-11-15 09:34:57.690164: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [8]:
#compiling the model
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#training model
epochs = 100  # Set the number of epochs
#this is taking a MINUTE --> should do in rivanna most likely
#ten epochs is not enough -- we are gonna need way more OR potench more data bc 
#this model is not accurate at all (about 0.1)

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

Epoch 1/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 117s 3s/step - accuracy: 0.0739 - loss: 3.3587 - val_accuracy: 0.0885 - val_loss: 2.5688
Epoch 2/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 110s 3s/step - accuracy: 0.0941 - loss: 2.5746 - val_accuracy: 0.0625 - val_loss: 2.5698
Epoch 3/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 106s 3s/step - accuracy: 0.0735 - loss: 2.5662 - val_accuracy: 0.1094 - val_loss: 2.6148
Epoch 4/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 109s 3s/step - accuracy: 0.0897 - loss: 2.5557 - val_accuracy: 0.1198 - val_loss: 2.5496
Epoch 5/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 114s 3s/step - accuracy: 0.1167 - loss: 2.5378 - val_accuracy: 0.1042 - val_loss: 2.5396
Epoch 6/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 107s 3s/step - accuracy: 0.1132 - loss: 2.5386 - val_accuracy: 0.0990 - val_loss: 2.5406
Epoch 7/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 107s 3s/step - accuracy: 0.1360 - loss: 2.5272 - val_accuracy: 0.0938 - val_loss: 2.5776
Epoch 8/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 109s 3s/step - accuracy: 0.1176 - loss: 2.5276 - val_accuracy: 0.

In [ ]:
#evaluating and printing accuracy
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")